In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool


# 步骤 1（可选）：添加一个名为 `my_image_gen` 的自定义工具。
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description` 用于告诉智能体该工具的功能。
    description = 'AI 绘画（图像生成）服务，输入文本描述，返回基于文本信息绘制的图像 URL。'
    # `parameters` 告诉智能体该工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '期望的图像内容的详细描述',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` 是由 LLM 智能体生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)


# 步骤 2：配置您所使用的 LLM。
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'qwen-max',
    'model_server': 'dashscope',
    'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
    'generate_cfg': {
        'top_p': 0.8
    }
}

llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'deepseek-v3',
    'model_server': 'https://dashscope.aliyuncs.com/compatible-mode/v1',
    'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
    'generate_cfg': {
        'top_p': 0.8
    }
}

# 步骤 3：创建一个智能体。这里我们以 `Assistant` 智能体为例，它能够使用工具并读取文件。
system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 首先绘制一幅图像，得到图像的url，
- 然后运行代码`request.get`以下载该图像的url，
- 最后从给定的文档中选择一个图像操作进行图像处理。
用 `plt.show()` 展示图像。
你总是用中文回复用户。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` 是框架自带的工具，用于执行代码。
import os
# 获取文件夹下所有文件
file_dir = os.path.join('./', 'docs')
files = []
if os.path.exists(file_dir):
    # 遍历目录下的所有文件
    for file in os.listdir(file_dir):
        file_path = os.path.join(file_dir, file)
        if os.path.isfile(file_path):  # 确保是文件而不是目录
            files.append(file_path)
print('files=', files)

bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=files)

# 步骤 4：作为聊天机器人运行智能体。
messages = []  # 这里储存聊天历史。
query = "介绍下雇主责任险"
# 将用户请求添加到聊天历史。
messages.append({'role': 'user', 'content': query})
response = []
current_index = 0
for response in bot.run(messages=messages):
    if current_index == 0:
        # 尝试获取并打印召回的文档内容
        if hasattr(bot, 'retriever') and bot.retriever:
            print("\n===== 召回的文档内容 =====")
            retrieved_docs = bot.retriever.retrieve(query)
            if retrieved_docs:
                for i, doc in enumerate(retrieved_docs):
                    print(f"\n文档片段 {i+1}:")
                    print(f"内容: {doc.page_content}")
                    print(f"元数据: {doc.metadata}")
            else:
                print("没有召回任何文档内容")
            print("===========================\n")
        #break

    current_response = response[0]['content'][current_index:]
    current_index = len(response[0]['content'])
    print(current_response, end='')
# 将机器人的回应添加到聊天历史。
#messages.extend(response)

files= ['./docs\\1-平安商业综合责任保险（亚马逊）.txt', './docs\\2-雇主责任险.txt', './docs\\3-平安企业团体综合意外险.txt', './docs\\4-雇主安心保.txt', './docs\\5-施工保.txt', './docs\\6-财产一切险.txt', './docs\\7-平安装修保.txt', './docs\\平安产险交通出行意外伤害保险（互联网版）产品说明.pdf', './docs\\平安产险交通工具意外伤害保险（互联网版）条款.pdf', './docs\\平安企业团体综合意外险(互联网版)适用条款.pdf', './docs\\平安商业综合责任保险（亚马逊）.pdf', './docs\\平安境内紧急医疗救援服务条款.pdf', './docs\\平安附加疾病身故保险条款.pdf']


2025-04-21 15:32:49,181 - memory.py - 113 - INFO - {"keywords_zh": ["雇主责任险", "雇主", "责任险", "保险"], "keywords_en": ["employer liability insurance", "employer", "liability insurance", "insurance"], "text": "介绍下雇主责任险"}
2025-04-21 15:32:50,144 - simple_doc_parser.py - 411 - INFO - Read parsed ./docs\1-平安商业综合责任保险（亚马逊）.txt from cache.
2025-04-21 15:32:50,148 - doc_parser.py - 108 - INFO - Start chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt)...
2025-04-21 15:32:50,150 - doc_parser.py - 126 - INFO - Finished chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt). Time spent: 0.0 seconds.
2025-04-21 15:32:50,160 - simple_doc_parser.py - 411 - INFO - Read parsed ./docs\2-雇主责任险.txt from cache.
2025-04-21 15:32:50,163 - doc_parser.py - 108 - INFO - Start chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt)...
2025-04-21 15:32:50,164 - doc_parser.py - 126 - INFO - Finished chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt). Time spent: 0.0 seconds.
2025-04-21 15:32:50,171 - simple_doc_parser.py - 4

雇主责任险是一种专门为企业设计的保险产品，旨在帮助企业转移因员工在工作期间发生意外事故或职业病而导致的赔偿责任。以下是雇主责任险的主要内容和特点：

### 1. **保障范围**
雇主责任险的保障范围包括：
- **死亡赔偿金**：员工因工作原因导致死亡，保险公司将支付相应的赔偿金。
- **伤残赔偿金**：员工因工作原因导致伤残，保险公司将根据伤残等级支付赔偿金。
- **医疗费用**：员工因工作原因受伤或患病，保险公司将承担相应的医疗费用。
- **误工费用**：员工因工作原因暂时无法工作，保险公司将支付误工费用。
- **法律诉讼费用**：企业因员工工伤事故引发的法律诉讼，保险公司将承担相关费用。

### 2. **适用对象**
雇主责任险适用于所有有用工需求的单位，包括企业、工厂、餐饮、建筑等各类行业。

### 3. **产品亮点**
- **雇主责任转嫁**：将企业对员工的风险责任转嫁给保险公司，减轻企业的经济负担。
- **降低用工风险**：通过保险赔偿，减少企业因员工工伤事故带来的经济损失。
- **减少工伤纠纷**：保险公司及时赔付，减少企业与员工之间的纠纷。
- **提高员工保障福利**：为员工提供更全面的保障，提升员工满意度和忠诚度。
- **税前列支**：企业购买雇主责任险的保费可以在企业所得税前扣除，享受税收优惠。

### 4. **产品版本**
雇主责任险通常有多个版本，如A款、B款和94款，不同版本在保障范围和赔偿标准上有所差异，企业可以根据自身需求选择合适的版本。

### 5. **与团体意外险的区别**
- **法律依据**：雇主责任险符合国家法律法规要求，而团体意外险主要体现企业对员工的福利，没有相关法律依据。
- **保障范围**：雇主责任险可以承保职业性疾病和误工费用，而团体意外险通常不包含这些保障。

### 6. **理赔案例**
例如，某工厂为员工购买了雇主责任险，员工在工作中不幸受伤致残，保险公司根据伤残等级支付了相应的赔偿金、医疗费用和误工费用，帮助企业减轻了经济负担。

### 7. **理赔流程**
雇主责任险的理赔流程包括线上和线下两种方式：
- **线上流程**：通过企业APP上传资料，等待审核，确认金额后支付赔款。
- **线下流程**：拨打客服电话，现场查勘，准备资料，确认理赔金额后支付赔款。

